# Decision Tree Classifier


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import f1_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier, BaggingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedShuffleSplit
from sklearn import tree 

Load datasets manuseados


In [ ]:
df_train = pd.read_csv('../datasets_manuseados/train_radiomics_hipocamp_cleaned.csv')
df_test = pd.read_csv('../datasets_manuseados/test_radiomics_hipocamp_cleaned.csv')

In [ ]:

# Separar features e target do treino
X = df_train.drop(columns=['Transition'])  # Drop da coluna target
y = df_train['Transition']  # Coluna target
X_test = df_test  # Dataset de teste (sem a coluna 'Transition')

# Dividir o dataset de treino para validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir o modelo base
dt = DecisionTreeClassifier(random_state=42)

# Grid de hiperparâmetros
param_grid = {
    'criterion': ['gini', 'entropy'],      # Critério de divisão
    'max_depth': [None, 5, 10, 20, 30],   # Profundidade máxima
    'min_samples_split': [2, 5, 10],      # Mínimo de amostras para dividir um nó
    'min_samples_leaf': [1, 2, 4],        # Mínimo de amostras em uma folha
    'max_features': [None, 'sqrt', 'log2']  # Máximo de features consideradas por nó
}

# Fazer GridSearchCV para encontrar os melhores parâmetros
grid_search = GridSearchCV(estimator=dt, param_grid=param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Melhor modelo encontrado
best_dt = grid_search.best_estimator_

# Identificar a importância das features
feature_importances = best_dt.feature_importances_
important_features = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

# Selecionar as features mais importantes (com importância acima da média)
mean_importance = feature_importances.mean()
selected_features = important_features[important_features['Importance'] > mean_importance]['Feature']

# Salvar as features importantes em um arquivo CSV
selected_features.to_csv('../check_files_hip/selected_important_features.csv', index=False)

# Ajustar os datasets de treino, validação e teste para incluir apenas as features selecionadas
X_train_selected = X_train[selected_features]
X_val_selected = X_val[selected_features]
X_test_selected = X_test[selected_features]

# Treinar o modelo com as features selecionadas
best_dt.fit(X_train_selected, y_train)

# Avaliar no conjunto de validação
val_accuracy = best_dt.score(X_val_selected, y_val)
print(f"Acurácia no conjunto de validação: {val_accuracy:.4f}")

y_pred = best_dt.predict(X_val_selected)
print("Relatório de classificação no conjunto de validação:")
print(classification_report(y_val, y_pred))

# Fazer previsões no dataset de teste
predictions = best_dt.predict(X_test_selected)

# Adicionar as previsões ao dataset de teste
df_test['Transition'] = predictions

# Salvar apenas as colunas 'RowID' e 'Transition' no dataset de teste com as previsões
df_test[['RowID', 'Transition']].to_csv('../submissions/dtc.csv', index=False)


Data Split and Grid Search and Predict